In [2]:
from datasets import load_dataset
import datasets
#dataset = load_dataset("yelp_review_full")
dataset = datasets.load_from_disk("/mnt/workspace/data/yelp_review_full_data")

In [3]:
dataset["train"][10]

{'label': 0,
 'text': "Owning a driving range inside the city limits is like a license to print money.  I don't think I ask much out of a driving range.  Decent mats, clean balls and accessible hours.  Hell you need even less people now with the advent of the machine that doles out the balls.  This place has none of them.  It is april and there are no grass tees yet.  BTW they opened for the season this week although it has been golfing weather for a month.  The mats look like the carpet at my 107 year old aunt Irene's house.  Worn and thread bare.  Let's talk about the hours.  This place is equipped with lights yet they only sell buckets of balls until 730.  It is still light out.  Finally lets you have the pit to hit into.  When I arrived I wasn't sure if this was a driving range or an excavation site for a mastodon or a strip mining operation.  There is no grass on the range. Just mud.  Makes it a good tool to figure out how far you actually are hitting the ball.  Oh, they are cash 

In [4]:
import random
import pandas as pd
import datasets
from IPython.display import display, HTML

In [5]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset["train"])

,label,text
0,3 stars,"The food is good, considering that this is Vegas where Vietnamese/Chinese food cannot compare to LA or OC. They serve in one size and have relatively large portions. \n\nService here is hit and miss. Sometimes, you get really good waiters and sometimes, you just get those that gives you attitude."
1,3 stars,"Definitely not for non meat eaters. We went for brunch so I just ordered some egg dish - I can't even remember what it was. \n\nHowever everyone else ordered chicken and waffles, mac and cheese and greens and was raving about it. That's all I have to say about that!"
2,5 stars,This place is super awesome! The service was great! Very attentive! The food was absolutely on point. Order the shrimp fajitas and they were delicious! Drinks were amazing and at a reasonable price! Homemade tortillas were delicious! Ugh loved it here! Would highly recommend this restaurant to my friends!
3,2 star,"I'm feeling a bit generous with this two star review. Both stars are basically for the fun layout and beautiful look of this club and the fact that we had a limited open bar until midnight. I've been to Encore Beach Club over half a dozen times and the staff has always been helpful and polite; apparently the same cannot be said for the night time hours when the space becomes Surrender. Took a group of seven girls here for a bachelorette and we were pretty poorly treated. After a night of being treated like royalty at Drai's, I left at like 11:30 because I was just tired of being treated like trash. A few tips- have the female host checking the guest list to make eye contact and not ignore large all girl groups (that's what you wanted in here), have your security not treat well behaved women like criminals, and have all your bartenders be consistent and helpful. You are NOT the nicest club in town anymore, Surrender. Maybe work for it a bit more."
4,2 star,"2.5 stars because even our guys got in free! It was on a Sat night. We arrived at around 1045p. Went straight to the Invited guest line which was only about a 5 min wait. The lady checked our names and then gave us pink tickets. Went down the escalator and to the entrance with ticket in hand and off we go. 2 girls and 2 guys, 50/50 ratio and we got in for free without much of a wait! Awesome!\n\nAs for the club itself...its alright. It is kinda like \""jet\"" in that the layout doesnt really have a large dance floor. Stripper pole in the center and mainly a wide aisle around to dance. It was only about 11pm and alraedy it was super crowded! Its vegas...how could it be so crowded already?! Music was pretty good that night. Lots of bars, maybe 3. When you walk into club you have to take the stairs down to the dance floor otherwise you head up to the catwalk/upstairs area and see really how crowded the dance floor is!\n\nthe only good thing about that whole evening there was the fact we got our guys in for free.\n\nDrinks were expensive as usual! $18 for a grey goose on the rocks. We asked for a cranberry chaser on the side. \n\nThe crowd was mixed with average looking. I didn't like how crowded it was and how difficult it was to find a place to stand or even dance! \n\nWould I go again? Only if girls AND guys get in free and we pre party it up so we don't have to pay the ridiculous drink prices."
5,4 stars,"This was my last meal before leaving the great city of Las Vegas. Some of the people in our group swore by it and I am so glad they made the recommendation. Like some of the other reviews, my only issue was the speed of service. But our waitress Patty was super cool and came back to check in on us pretty often.\n\nWould love to experience this place after a long night out on the Strip, maybe next trip..."
6,4 stars,"I was driving down Spring Mountain on my way home and decided to stop by Lee's for a quick milk tea, when I saw the sign for Fat Dumpling. I had never noticed this place pop up, so stopped in for a quick to go order. \n\nThey have a small dining area, 

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/mnt/workspace/model/bert_base_cased",local_files_only=True)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 50000/50000 [00:18<00:00, 2758.07 examples/s]


In [8]:
show_random_elements(tokenized_datasets["train"], num_examples=1)

,label,text,input_ids,token_type_ids,attention_mask
0,5 stars,"I am in nostalgic heaven right now! I really love these old diners and the ambiance they have! The food here is really good! We had the biscuits and gravy (fluffy and delicious with good flavor!), waffle combo (the waffles are awesome!!!), denver omelet and home fries (which are both worth having), and to top it off; a cinnamon roll! Holy cow!! We ate like kings, on not-a-king's budget!! 100% satisfaction! :)","[101, 146, 1821, 1107, 1185, 19760, 11007, 9775, 1268, 1208, 106, 146, 1541, 1567, 1292, 1385, 20162, 1116, 1105, 1103, 1821, 12324, 2093, 1152, 1138, 106, 1109, 2094, 1303, 1110, 1541, 1363, 106, 1284, 1125, 1103, 16516, 1116, 10182, 6439, 1105, 176, 1611, 7170, 113, 23896, 12383, 1105, 13108, 1114, 1363, 16852, 106, 114, 117, 20049, 13327, 27481, 1186, 113, 1103, 20049, 25754, 1132, 14918, 106, 106, 106, 114, 117, 10552, 4121, 184, 10212, 2105, 1105, 1313, 175, 3377, 113, 1134, 1132, 1241, 3869, 1515, 114, 117, 1105, 1106, 1499, 1122, 1228, 132, 170, 172, 23339, 5155, 106, 3930, 13991, ...]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]"


In [9]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("/mnt/workspace/model/bert_base_cased",local_files_only=True, num_labels=5)

In [13]:
from transformers import TrainingArguments

model_dir = "models/bert-base-cased"

# logging_steps 默认值为500，根据我们的训练数据和步长，将其设置为100
training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("/mnt/workspace/evaluate/metrics/accuracy")

2024-01-15 17:48:16.499455: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-15 17:48:17.582300: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [14]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir=f"{model_dir}/test_trainer",
                                  evaluation_strategy="epoch", 
                                  logging_dir=f"{model_dir}/test_trainer/runs",
                                  logging_steps=100)

In [24]:
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_always_push=False,
hub_mod

In [16]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

Detected kernel version 4.19.24, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [17]:
! nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Jan 15 17:54:53 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:09.0 Off |                    0 |
| N/A   37C    P0    55W / 300W |   1461MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
! nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Jan 15 17:58:28 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:09.0 Off |                    0 |
| N/A   39C    P0    55W / 300W |   7761MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.833000,1.250888,0.529000
2,0.537800,1.406651,0.551000
3,0.402800,1.530191,0.578000


TrainOutput(global_step=375, training_loss=0.5285118408203126, metrics={'train_runtime': 123.9937, 'train_samples_per_second': 24.195, 'train_steps_per_second': 3.024, 'total_flos': 789354427392000.0, 'train_loss': 0.5285118408203126, 'epoch': 3.0})

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.833000,1.250888,0.529000
2,0.537800,1.406651,0.551000
3,0.402800,1.530191,0.578000


TrainOutput(global_step=375, training_loss=0.5285118408203126, metrics={'train_runtime': 126.1818, 'train_samples_per_second': 23.775, 'train_steps_per_second': 2.972, 'total_flos': 789354427392000.0, 'train_loss': 0.5285118408203126, 'epoch': 3.0})

In [19]:
small_test_dataset = tokenized_datasets["test"].shuffle(seed=64).select(range(100))

In [20]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.9284188747406006,
 'eval_accuracy': 0.47,
 'eval_runtime': 1.0732,
 'eval_samples_per_second': 93.178,
 'eval_steps_per_second': 12.113,
 'epoch': 3.0}

In [21]:
trainer.evaluate(small_test_dataset)

{'eval_loss': 1.9284188747406006,
 'eval_accuracy': 0.47,
 'eval_runtime': 1.0854,
 'eval_samples_per_second': 92.132,
 'eval_steps_per_second': 11.977,
 'epoch': 3.0}

In [22]:
trainer.save_model(f"{model_dir}/finetuned-trainer")

In [23]:
trainer.save_state()